In [14]:
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

import matplotlib.pyplot as plt
import numpy as np

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [3]:
# use tf.data to batch and shuffle the dataset
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [4]:
# model subclassing
class MyModel(Model):
  # define layers of the model
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation="relu")
    self.flatten = Flatten()
    self.d1 = Dense(128, activation="relu")
    self.d2 = Dense(10)

  # feed forward
  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

In [5]:
# create instance of the model
model = MyModel()

In [6]:
# define loss function and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [7]:
# select metrics to measure the loss and the accuracy of the model
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [8]:
# what is @tf.function

In [9]:
# Use tf.GradientTape to define function that trains the model
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g Dropout)
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)   # defined loss function in one of the above cell
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  # accumulating loss and accuracy metrics
  train_loss(loss)
  train_accuracy(labels, predictions)
  

In [10]:
# define function that tests the trained model
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout)
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  # accumulating loss and accuracy metrics for testing step
  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [11]:
# Train!
EPOCHS = 4

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(f'Epoch {epoch+1}, Loss: {train_loss.result()*100}, Accuracy: {train_accuracy.result()}, Test Loss: {test_loss.result()}, Test Accuracy: {test_accuracy.result()*100}')


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 13.636896133422852, Accuracy: 0.958816647529602, Test Loss: 0.060373950749635696, Test Accuracy: 98.05999755859375
Epoch 2, Loss: 4.253469944000244, Accuracy: 0.9866333603858948, Test Loss: 0.056468915194272995, Test Accuracy: 98.23999786376953
Epoch 3, Loss: 2.2810616493225098, Accuracy: 0.9924666881561279, Test Loss: 0.056078098714351654, Test Accuracy: 98.23999786376953
Epoch 4, Loss: 1.3133422136306763, Accuracy: 0.9956833124160767, Test Loss: 0.060645412653684616, Test Accuracy: 98.3

In [24]:
for test_images, test_labels in test_ds:
  prediction = model.predict(test_images, batch_size=32)
  for i, p in enumerate(prediction[:5]):
    print(f"predicted: {np.argmax(p)}\tactual: {test_labels[i]}")
  break


predicted: 7	actual: 7
predicted: 2	actual: 2
predicted: 1	actual: 1
predicted: 0	actual: 0
predicted: 4	actual: 4
